In [6]:
import numpy as np
import pprint
from operator import itemgetter
from mdp_matrix import GridWorld
from sarsa import sarsa
from expected_sarsa import expected_sarsa
from double_sarsa import double_sarsa
from double_expected_sarsa import double_expected_sarsa

In [2]:
test_rewards = [[i, j, -1.0] for i in range(5) for j in range(5)]
# test_rewards[2] = [0, 2, 1]
test_rewards[23] = [4, 3, 1]
# test_rewards = [[0, 3, 5],
#                 [0, 1, 10]]
terminal_states = [23]
print test_rewards
gw = GridWorld(5, test_rewards, terminal_states)

[[0, 0, -1.0], [0, 1, -1.0], [0, 2, -1.0], [0, 3, -1.0], [0, 4, -1.0], [1, 0, -1.0], [1, 1, -1.0], [1, 2, -1.0], [1, 3, -1.0], [1, 4, -1.0], [2, 0, -1.0], [2, 1, -1.0], [2, 2, -1.0], [2, 3, -1.0], [2, 4, -1.0], [3, 0, -1.0], [3, 1, -1.0], [3, 2, -1.0], [3, 3, -1.0], [3, 4, -1.0], [4, 0, -1.0], [4, 1, -1.0], [4, 2, -1.0], [4, 3, 1], [4, 4, -1.0]]


In [4]:
[sarsa_Q, sarsa_avg_reward] = sarsa(gw, 100000)
print sarsa_avg_reward

-5.74045


In [5]:
[expected_sarsa_Q, expected_sarsa_avg]  = expected_sarsa(gw, 100000)
print np.reshape(np.argmax(expected_sarsa_Q, 1), (5,5))
print expected_sarsa_avg

[[0 1 0 0 1]
 [0 0 0 0 3]
 [0 0 0 0 0]
 [1 1 0 0 3]
 [1 1 1 0 3]]
-5.67772


In [7]:
[double_sarsa_Q, double_sarsa_avg]  = double_sarsa(gw, 100000)
print double_sarsa_avg

-5.74911


In [8]:
[double_expected_sarsa_Q, double_expected_sarsa_avg]  = double_expected_sarsa(gw, 100000)
print double_expected_sarsa_avg

-5.69206
